<img src="../AUXFILE/Reliance-logo.jpg" width="300" align="center">

# LiCSBAS on Etna Volcano 
### The current workflow is a run of LiCSBAS code to extract the mean ground velocity map and the associated time-series of displacement, of the post eruption phase relevant to the December 2018 event on Etna volcano

### Data are available from **COMET-LiCS** Sentinel-1 *InSAR* portal: 


<img src="../AUXFILE//COMET_Horizontal_Colour-300x113.jpg" width="250" align="up">

<https://comet.nerc.ac.uk/COMET-LiCS-portal/>

## setup of environment variables fo LiCSBAS execution and missing libs 

In [ ]:
import sys, os
from IPython.display import Image

In [ ]:
# setup of evnironment variables
os.environ ['LICSBAS_PATH']='/home/jovyan/MYLICS/LiCSBAS-master'
a = os.getenv('PATH')
b = os.getenv('LICSBAS_PATH')
c = os.getenv('PYTHONPATH')
os.environ ['PATH'] = f'{a}:{b}/bin'
d = f'{b}/LiCSBAS_lib'
os.environ ['PYTHONPATH'] = f'{c}:{d}'

In [ ]:
#install missing libs
! pip install astropy

### check LiCSBAS installation

In [ ]:

! LiCSBAS_check_install.py

## create processing folder with the name of the selected frame

In [ ]:
os.chdir('/home/jovyan/MYLICS/')

In [ ]:
frame = '124D_05291_081406' # descending set on Etna volcano
os.mkdir(frame)

In [ ]:
os.chdir(frame)
os.getcwd()

## STEP 01 - Inteferograms download

In [ ]:
#dowload the interferograms from COMET database
! LiCSBAS01_get_geotiff.py -s 20190101 -e 20211231

## STEP 02 - Multi-looking and data conversion

In [ ]:
# multi-looking and file format conversion
! LiCSBAS02_ml_prep.py -i GEOC -o GEOCml2 -n 2 

## STEP 04&03 - Optonal steps for atmo correction wiht GACOS and masking undesired areas

In [ ]:
# these steps have been skipped
# LiCSBAS03op_GACOS.py
# LiCSBAS04op_mask_unw.py

## STEP 05 - Clip data over the AOI

In [ ]:
#clip data around Etna volcano
! LiCSBAS05op_clip_unw.py -i GEOCml2 -o GEOCclip -g 14.7/15.3/37.44/37.95
# and check the data clipped...

In [ ]:
Image(filename='GEOCclip/hgt.png') 

## STEP 11 - Quality check

In [ ]:
# quality check
#! LiCSBAS11_check_unw.py -h
! LiCSBAS11_check_unw.py -d GEOCclip -c 0.1 -u 0.3

In [ ]:
# check network after step 11
Image(filename='TS_GEOCclip/network/network11.png')

In [ ]:
# check outputs of step 11
# list of bad ifg
!  head -n 10 TS_GEOCclip/info/11bad_ifg.txt

In [ ]:
# ... and visualize some of them: change the ifg pair!!!
Image(filename='TS_GEOCclip/11bad_ifg_ras/20210128_20210209.unw.png')

## STEP 12 - Loop closure analysis

In [ ]:
# loop closure analysis
!LiCSBAS12_loop_closure.py -d GEOCclip -t TS_GEOCclip -l 1.5 --multi_prime

In [ ]:
# check outputs of step 12
# list of bad ifg
!  head -n 10 TS_GEOCclip/info/12bad_ifg.txt

In [ ]:
# ... and visualize some of them: change the ifg pair!!!
Image(filename='TS_GEOCclip/12bad_ifg_ras/20200209_20200316.unw.png')

In [ ]:
#check triplet too ...
! ls TS_GEOCclip/12loop/bad_loop_png/

In [ ]:
# ... and visualize some of them: 
Image(filename='TS_GEOCclip/12loop/bad_loop_png/20200122_20200128_20200215_loop.png')

In [ ]:
# check network after step 12
Image(filename='TS_GEOCclip/network/network12.png')

## STEP 13 - SVD inversion

In [ ]:
# SVD inversion
! LiCSBAS13_sb_inv.py -d GEOCclip -t TS_GEOCclip 
# check network after step 13
Image(filename='TS_GEOCclip/network/network13.png')

In [ ]:
# and see the first estimation of velocity
Image(filename='TS_GEOCclip/results/vel.png')

## STEP 14 - Estimate of Standard deviation of veloctites

In [ ]:
# bootstrap for Velocity error estimation
! LiCSBAS14_vel_std.py -t TS_GEOCclip

In [ ]:
# and see the estimation of velocity error
Image(filename='TS_GEOCclip/results/vstd.png')

## STEP 15 and 16 - Masking and Filtering
### these two steps can be executed several times changing the input parameters, in order to identify the best final solution

In [ ]:
# masking time serires
! LiCSBAS15_mask_ts.py -h

In [ ]:
# masking time serires
! LiCSBAS15_mask_ts.py -t TS_GEOCclip -c 0.3 -u 1.5 -v 50 -T 1 -g 10 -s 5 -i 50 -l 5 -r 3
Image(filename='TS_GEOCclip/mask_ts.png')

In [ ]:
# filtering!
! LiCSBAS16_filt_ts.py -t TS_GEOCclip -s 2 --hgt_linear --hgt_min 500 --hgt_max 2000
Image(filename='TS_GEOCclip/results/vel.filt.mskd.png')

## Create my own map of velocity wiht preferred colorbar and velo ranges

In [ ]:
# create my own map of velocity
! LiCSBAS_disp_img.py -i TS_GEOCclip/results/vel.filt.mskd -p  TS_GEOCclip/info/EQA.dem_par -c jet --cmin -40 --cmax 40 --png velo_final.png
Image(filename='velo_final.png')

## Packing the final results 

In [ ]:
# zip the final h5 file 
! tar -czf Etna_results.tar.gz TS_GEOCclip/cum_filt.h5
print('DONE')